In [1]:
import roslibpy

client = roslibpy.Ros(host='robot-mb-01', port=9090)
client.run()
print('Is ROS connected?', client.is_connected)

Is ROS connected? True


In [2]:
first_goal = {}
robot_pose = {}

In [3]:
def set_robot_pose(pose):
    global robot_pose
    robot_pose = pose

In [4]:
robot_listener = roslibpy.Topic(client, '/robot_pose', 'geometry_msgs/PoseStamped')
robot_listener.subscribe(lambda message: set_robot_pose(message['pose']))

In [20]:
first_goal = robot_pose
print(robot_pose)

{'position': {'y': 0.045216232053563336, 'x': 0.19990246512275123, 'z': 0.1016}, 'orientation': {'y': 0.0, 'x': 0.0, 'z': 0.10979309351010794, 'w': 0.9939544640563172}}


In [21]:
from roslibpy.actionlib import Goal, ActionClient

action_client = ActionClient(client,
                             '/move_base',
                             '/move_base_msgs/MoveBaseAction')

goal = Goal(action_client,
            roslibpy.Message({ 'target_pose': {'header': { 'frame_id': 'map' },
                             'pose': first_goal }}))

In [22]:
goal.on('result', lambda f: print('Pose reached'))

<function __main__.<lambda>(f)>

In [23]:
goal.send()
result = goal.wait(10)
action_client.dispose()
print('Result: {}'.format(result))

Exception: Goal failed to receive result

In [19]:
robot_listener.unsubscribe()